In [14]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
driver.get("https://www.idus.com/c/category/3") #tab[0]
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
category = soup.findAll('li',{'class':'side_nav__menu'})



In [22]:
bId=0
sId=0
bigCategory=[]
smallCategory=[]
smallCategoryName=[]
for big in category:
    bId+=1
    bname=big.find("a").text
    bigCategory.append([bId,bname])
    small=big.findAll("li",{"class":"side_nav__submenu"})
    for s in small:
        sId+=1
        sname=str(s.text.strip())
        smallCategoryName.append(sname)
        smallCategory.append([sId,bId,sname])
    

In [16]:
with open ("ourdus_offline_class_big_category.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    myWriter.writerow(["big_category_id","big_category_name"])
    for i in range(len(bigCategory)):
        myWriter.writerow(bigCategory[i])

with open ("ourdus_offline_class_small_category.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)
    myWriter.writerow(["small_category_id","big_category_id","small_category_name"])
    for i in range(len(smallCategory)):
        myWriter.writerow(smallCategory[i])
        

In [40]:
import time
import re
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
from selenium.webdriver import ActionChains

driver = webdriver.Chrome('C:/Users/KSY/Desktop/chromedriver.exe')
# driver.get("https://www.idus.com/c/latest") #tab[0]
driver.execute_script('window.open("about:blank", "_blank");') #창 1개더 열어놓기 tab[1]
tabs = driver.window_handles
# time.sleep(3)

def scroll():
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# driver.switch_to_window(tabs[0])
# scroll()
# html = driver.page_source
# soup = BeautifulSoup(html, 'lxml')
# classlist = soup.findAll('li',{'class':'ui_grid__item'}) #모든 클래스 목록


data=[] #클래스 data 1개씩
# class_id , author_id,small_category_id, class_name, class_price, class_description, class_hit, class_purchase,
# class_duration, class_level, class_place, user_max, class_like, class_rate

#image_id,class_id,class_img

class_id = 0
author_id=0
imageList=[]
cnt=0

#클래스 한개씩 작업
for row in classlist:   
    cnt+=1
    if cnt==100: #오프라인 클래스 100개로 제한
        break
        
    author_id+=1
    class_id+=1
    
    # 현재 클래스의 상세 페이지 URL
    info_URL = row.find('a',{"class":"ui_card__img"})
    info_URL = info_URL["href"]
    info_URL = "https://www.idus.com/" + str(info_URL)

    driver.switch_to_window(tabs[1]) #두번째 탭으로 이동
    driver.get(info_URL)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml');
    
    class_name = soup.find('h2',{"class":"sticky_aside__title"}).text
    class_price = soup.find("strong",{"class":"sold-price"}).text
    
    
    #이미지들
    images = soup.find('ul',{'class':'img-list indicator'})
    imageList.append([class_id,images])

    #난이도, 소요시간,최대인원
    basic_info = soup.find('div',{'class':'def_group'})
    basic_info = basic_info.findAll('dd')
    class_level = basic_info[0].text
    class_duration=basic_info[1].text
    user_max=basic_info[2].text

    class_description = soup.find('p',{'class':'article_block__para'}).text
    
    class_place = soup.find('div',{'class':'map__labelbox'}) 
    class_place = class_place.find('strong').text
    
    hashtag = soup.find('div',{'class':'hashtag-box'})
    small_category = hashtag.find('a',{'class':'txt-strong'}).text.strip()
    
    if small_category in smallCategoryName:
        small_category_id=smallCategoryName.index(small_category)+1
    else:
        small_category_id=35
    
    
    ##조회수, 별점,구매수
    class_hit=0
    class_rate=0
    class_purchase=0
    class_like=0
    
    # class_id , author_id,small_category_id, class_name, class_price, class_hit, class_purchase,
    # class_duration, class_level, class_place, user_max, class_like, class_rate,class_description
    data.append([class_id,author_id,small_category_id,class_name,class_price,class_hit,class_purchase,
                 class_duration,class_level,class_place,user_max,class_like,class_rate,class_description])
    print([class_id,author_id,small_category_id,class_name,class_price,class_hit,class_purchase,
                 class_duration,class_level,class_place,user_max,class_like,class_rate,class_description])
  

<ipython-input-40-54c8a5d3ee1c>:64: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) #두번째 탭으로 이동


[1, 1, 12, '정성스럽고 격식있는 보자기 포장 매듭법 !', '80,000', 0, 0, '1.5 시간', '중', '경기 성남시', '최대 4명', 0, 0, '<  KNOTIST _ 노티스트 / 원데이 클래스  >\n\nº 1인 기준 : 8만원 / 매듭법 4가지\n(재료비 포함)\n\n\nº수업 시간 : 약 1시간 30분 \n(클래스 특성상 수업 시간이 더 소요될 수 있습니다.)\n\n\n- 보자기와 박스 및 재료들 4가지씩 제공됩니다.\n따로 준비하실 재료는 필기도구만 가져오시면 됩니다. !\n\n- 수업은 상시모집으로 일정 조율 후 진행됩니다.\n\n_____________________________\n\n\n노티스트 - 원데이 클래스에는\n\nº 다양한 보자기 색으로 여러 가지 매듭법을 배울 수 있으며, 의미가 담긴 선물에 정성과 격식을 담아 포장하실 수 있습니다.\n\nº 명절, 기념일 및 답례품과 같은 유용한 선물 포장법을 배워보고자 합니다.\n\nº 물건에 구애를 받지 않고 어떤 선물이든 격식있게 포장할 수 있습니다. ']
[2, 2, 12, '수제 전통계절주를 집에서 간단하게 빚어 건강하게 즐기자!', '48,000', 0, 0, '2 시간', '하', '서울 종로구', '최대 6명', 0, 0, '전통 발효 계절주 수업을 진행합니다.\n\n먼저 기초 이론 설명과 함께 시연을 하고...본인이 직접 만들어 보시고 ..완성품을 가져 가시는 방법입니다.\n\n1. 준비과정과 재료 설명\n\n2. 고두밥 짓기\n\n3. 직접 술빚기\n\n4. 완성 후 용기에 담기\n\n']
[3, 3, 32, '[3월의 데일리솝] 솜사탕컬러로 만드는 팜프리 비건비누 만들기', '50,000', 0, 0, '2 시간', '중', '서울 마포구', '최대 2명', 0, 0, '혹시 비건비누에 대해 알고 계신가요?\n\n팜프리비누라고도 불리며 비누를 만들면서 필수로 들어가는 팜유라는 식물성오일이 들어가지 않은 비누를 말해요.\n\n팜유는 비누의 단단함과 조밀한 거

[13, 13, 21, '치명적인 귀여움 [ 미니 페이퍼 다육이 ] 클래스 ', '25,000', 0, 0, '2 시간', '하', '인천 부평구', '최대 4명', 0, 0, '[ 미니 페이퍼 다육이 ] 만들기 *\n\n1. 2가지 타입의 화분 중 1가지를 선택합니다.\n2. 그린컬러 or 원하시는 다른 컬러의 주름지를 이용하여 도안대로 잘라줍니다.\n3. 기법을 이용하여 잎의 모양을 잡아주고, 붙여줍니다.\n4. 화분에 디스플레이 한 후 마무리 작업을 한 후, 사진 촬영 및 포장을 합니다.\n\n소요시간은 총 2시간 소요됩니다.']
[14, 14, 32, '커스텀대로 만드는 반려견 수제비누', '50,000', 0, 0, '1 시간', '중', '경기 남양주시', '최대 6명', 0, 0, '\n저는 수제비누로 애들 목욕시키고부터\n피부병으로 인해 병원 가는 일이 더 이상 없어졌어요\n👋🏻👋🏻👋🏻\n\n\n반려견 뿐만 아니라 저희 가족도 수제비누를 사용하는데,\n정말 건조함을 느끼지 못 할 뿐더러\n샤워시간도 단축되고 여러모로 장점이 많은 것 같아요👶🏻\n\n\n비누는 단단히 굳히는 30일정도의 시간이 필요해요\n다 만든 비누는 틀에서 빼내어 조각 낸 후,\n자택으로 배송됩니다 :)\n비누가 굳어가며 은은히 풍기는 아로마향으로 방 안이 가득 채워질거에요.\n사용감도 좋지만 방향으로도 좋은 수제비누,\n\n\n반려견 비누와 견주님 비누를 함께 만들 시, 추가 3만원 입니다🛀🏻\n\n\n']
[15, 15, 10, '[성수동데이트]나만의 도마만들기 도마원데이클래스', '68,000', 0, 0, '2.5 시간', '중', '서울 성동구', '최대 4명', 0, 0, '정근날 도마 원데이 클래스\n목공예를 처음 시작하시는분들도 부담없이 \n시작하실 수 있는 클래스입니다 :)\n\n기본적인 나무의 성질을 이해하고  \n원하는 디자인으로 제작하는 과정을 통해 \n목업 프로세스를 경험하실 수 있습니다.\n\n아이디어스 도마 수강 프로세스\n\n· 1: 수종(화이트오크

[24, 24, 6, '[홍대/연남] 엄마와 딸이 함께 만드는 가죽 도장케이스,립밤케이스', '37,000', 0, 0, '2 시간', '하', '서울 마포구', '최대 3명', 0, 0, '아노말리는 연남동과 끝자락 부근에 위치한\n소규모 공방 및 쇼룸으로 천연베지터블 가죽을 주소재로 \n주문제작과 클래스를 진행하기도 하고 \n제가 함께 지내는 강아지들과 출근해 \n작업하는 공간이기도 한데요 :)\n평소 보물찾기를 하듯 골목골목 \n소소하고 특색있는 공간을 찾아다니시는 것을 \n좋아하시는 분들이라면 주변골목도 구경하시고 \n잠시 이곳에 머물러 저와 함께 가죽에 손바느질을 하며 \n천천~히 흘러가는 시간을 즐겁게 채워가시는건 어떠실까요?\n\n아노말리로 가죽하러오세요~! :)\n\n----------------------------------\n\n1단계) 재료 소개 및 도구 사용설명\n미리 재단 해 놓은 가죽 위에 구멍을 내어 줄 도구들과 \n바느질 구멍을 내는 방법에 대해 안내받으신 후 타공.\n\n2단계) 실 색상 선택\n개인의 취향에 맞는 실 색상을 선택합니다.\n\n3단계) 바느질 설명 및 바느질\n제품에 해당하는 바느질 기법을 안내 해 드립니다.\n\n4단계) 단면마감\n패턴에 따라 단면마감을 2단계에서 \n진행하게 될 수도 있지만 \n보통은 마지막 단계에서 마감을 합니다.\n\n\n※ 각인추가 미포함가격입니다. ※\n\n-------------------------------------\n\n- 일행 외 다른팀과의 수업을 동시 진행하지 않습니다.  \n  (단 한분만 예약하셔도 클래스는 진행됩니다.)\n  그 시간은 오롯이 예약자님 혹은 예약자님과 \n  함께 하시는 분의 소중한 시간이에요.\n  아노말리는 프라이빗클래스로 운영됩니다. :)\n\n\n- 수업은 최대 3:1 수업까지 가능합니다.\n\n- 개인차가 있지만 완성까지 최대 2시간정도의 시간이 소요 되실 수 있습니다.\n\n- 웰컴드링크, 약간의 주전부리가 준비되어 있습니다.\n\n\n사용

[38, 38, 7, '법정순은925 은볼 비즈반지 + 순은체인반지 제작 (후기서비스반지!!!)', '38,000', 0, 0, '1 시간', '하', '서울 마포구', '최대 4명', 0, 0, '스케줄을 미리 열어놓지않아요 ^^ 시간상담후 스케줄 열어놓으니 \n메세지로 시간 상담 부터 해주세요 ^^ 되도록이면  맞춰드려요 ~~*\n\n순은이라고 하면은 함량이 99.9% 임을 의미해요. \n"은"은 금속 중에서 상당히 무른 편이예요\n이렇게 순은은 실용성이 조금 떨어지기 때문에 \n강한 금속을 섞어서 만든게 925실버에요.\n\n925 실버, Sterling Silver, 법정순은은 함량 92.5%\n실버 925라고 불리는 이유는\n은이 92.5% 함유돼있기 때문인데요,\n\n그래서 이번에 진행할 클래스는 법정순은볼로 \n다양한 팬던트로 비즈반지를 만들어봅니다\n데일리반지로도 엄청나게 고객분이 찾고 있어서 관심이 생겨서 만든 클래스 입니다 \n주로 친구분들, 가족, 언니 동생 우정반지로 서로의추억과 특별한추억을 쌓기 위해 많이 찾아주십니다\n여러차례 원데이클래스운영 경력 있어요 !!!!!\n\n\n진행순서\n-도구와재료 사용법 소개 본인에 맞는 손가락 호수 상담(5분)\n-본인의취향에 맞는 팬던트 선택(5분)\n-법정순은925 은볼 비즈 + 체인반지 제작(30분)\n-제작완성후 사진촬영 하기 (5분)']
[39, 39, 5, '투웨이 네트백 클래스! 마크라메 가방', '50,000', 0, 0, '2.5 시간', '하', '서울 은평구', '최대 3명', 0, 0, '나를 사랑하게 되는 시간 손끝 감성 힐링 \n스튜디오 동그란입니다. \n\n동그란 매듭으로 무궁무진한 마크라메 세계에 초대합니다. \n\n마크라메는 매듭으로 섬유를 만드는 아라비아 전통 매듭공예랍니다. \n특별한 도구 없이 오직 나의 손으로 매듭을 지어 무궁무진한 \n마크라메 작품을 만들 수 있습니다. \n\n******************************************\n\n투웨

[49, 49, 7, '웨딩헤어악세사리 신부옆꽂이 혼주뒤꽂이 만들기 원데이클래스', '65,000', 0, 0, '3.5 시간', '하', '인천 남동구', '최대 2명', 0, 0, '웨딩헤어악세사리\n신부 옆꽂이와 혼주 뒤꽂이 각각 1개씩을 만들어가실 수 있는 클래스입니다.\n\n처음 도전하시는 분들도 어렵지 않게 만드실 수 있도록\n쉽고 재미있게 알려드립니다.\n각자의 취향대로 화려하게 또는 우아하게 완성해내는\n웨딩 헤어 악세사리  원데이클래스.\n\n퀄리티 높은 부자재 사용으로\n만족도가 높은 웨딩헤어악세사리 클래스는\n\n- 신부옆꽂이 1개, 혼주뒤꽂이 1개 과정\n- 수강시간 약 3~3.5시간\n- 1:1 지인동반시 최대 2:1수업\n- 수강일 최소 3일이전 수강예약\n- 마스크착용 필수, 체온 37.5도 이상 공방입실 불가\n\n클래스 수강가능 요일과 시간은 임의로 지정해두었습니다.\n메시지로 별도 문의해주시기 바랍니다.']
[50, 50, 14, '"달콤 쫀득"마카롱 만들기', '42,000', 0, 0, '2 시간', '중', '인천 부평구', '최대 3명', 0, 0, '  마카롱 체험 클래스 입니다(레시피 제공X)\n 미리 계량 해 논 재료로 전 과정 직접 만들어 포장까지 마무리~~~\n 프렌치 머랭 기법을 사용하며 꼬끄의 건조 시간이 짧은 (10~15분 이내) \n 일명 무건조 마카롱입니다\n\n 단축 시간만큼 더 많은 마카롱을 만들 수 있는 장점이 있으며\n 바닐라.초코.말차.얼그레이.오레오 필링 중 1개 직접 필링 맛을 선택하고 \n 30개 꼬끄짜서  짝 맞춰 샌딩 후 15개 마카롱을 완성 하실 수 있습니다\n 박스포장 or opp봉투포장 가능합니다(데코용 스프링클 무료제공)\n\n 가족,친구,연인 이색데이트 및 취미, 자기계발 원하시는 분 들, 초보자분들도, \n 창업을 원하시는 분들도 누구나 쉽게 따라 하 실수 있습니다  \n 코로나로인해  일행단독 최소1명~최대3명 프라이빗하게 진행됩니다\n (소독 및 손씻기 마스크착용 발열체크

[64, 64, 32, '주방비누 만들기', '60,000', 0, 0, '2 시간', '하', '강원 인제군', '최대 3명', 0, 0, '1. 클래스 접수\n2. 접수된 사항을 바탕으로 작가가 메세지를 통해 만들고 싶은 작품에 상세내용 및 날찌,시간 상담\n (***클래스 예약시 스케쥴 등록된 날짜,시간 외에도 클래스 가능하오니 접수 후 원하는 시간 메세지 주세요~)\n3. 예약일에 공방에서 수업진행\n4. 2주 보온 및 건조 후 공방에서 픽업 또는 택배 수령.   \n\n비누 1kg 기준입니다. ( 컷팅시 원하는 용량으로 컷팅 가능합니다)\n']
[65, 65, 32, '[데일리솝 시리즈] 동백오일을 넣은 제주동백비누 만들기', '50,000', 0, 0, '2 시간', '중', '서울 마포구', '최대 2명', 0, 0, '2020년을 맞아 처음 맞는 어색하고 정신없는 1월을 보내고 벌써 한달이 지나갔네요.\n\n2월에는 어떤 비누를 올릴까 고민하다가 겨울에만 피는 제주의 동백꽃과 오름을 표현한 제주동백비누를 준비해 보았어요.♡\n\n제주동백비누의 주성분은 동백오일과 식이유황 분말이에요. 동백오일은 끈적임이 없고 흡수가 빠른 오일로 헤어제품에도 많이 사용되는 식물성 오일이에요.\n\n피부에 사용시 모공을 막지않아 트러블 피부에도 좋아서 요즘처럼 마스크를 생활화하며 생기는 피부관리에 도움을 줍니다. \n\n함께 들어가는 식이유황 분말은 식용으로도 많이들 알고 계시겠지만 비누에 넣어서 사용히 염증과 진정에 도움을 주어서 동백오일과 시너지 효과를 내어줄거에요.\n\n식이유황분말은 비누액에 넣을때 잘 녹여서 첨가해야 하는데 그 방법을 잘 짚어드리면서 진행하도록 할게요.\n\n비누의 디자인은 제주를 가고픈 마음을 담아 제주오름과 청명한 하늘, 흰구름을 표현해 보았어요. 그 위에 동백꽃을 하나씩 올려서 마무리해 주었답니다.\n\n비누에 들어가는 색소는 최대한 천연분말만을 사용하여 만들어 보았어요. 하늘의 색은 저는 청명한 하늘색을 표현했지만 원하시는 색감으로도 

[78, 78, 6, '가죽과 함께 위드유레더 [필통 만들기] ', '80,000', 0, 0, '3 시간', '하', '인천 연수구', '최대 4명', 0, 0, '- 진행 순서\n1. 원데이에서 사용할 가죽에 대하여 알아보기\n2. 패턴에 대한 이해\n3. 기초 도구의 사용방법 익히기\n4. 기본 바느질 방법 익히기\n5. 단면 마감하기\n6. 인증샷 남기기\n\n- 스캐줄\n옵션에 없는 시간대를 원하시는 분들은 \n편히 메세지 보내주세요~\n공방 오픈 시간은 오전10:30~ 오후8:30 입니다.\n(화요일은 공방 휴무입니다.)\n\n- 가죽은 이태리 베라펠레 협회에서 인증 받은 \n베지터블 가죽 "푸에블로"를 사용합니다.\n색상은 총 브라운, 바이올렛, 올리브, 머스터드, 올모 총 5가지 입니다.\n\n푸에블로 가죽은 밀링타입의 무늬가 없는 가죽에\n쇠구슬을 굴려 가공한 가죽으로\n쇠구슬 무늬가 빈티지하게 나타나 있어 처음엔 무광이었다가\n사용하심에 따라 오일감이 올라와 색이 어두워지고, 매끈해지는\n반전매력이 뚜렷한 가죽입니다.\n\n저의 최애 가죽이기도 한 푸에블로는\n다른 가죽과는 다르게 태닝과 에이징속도가 빨라\n색이 급변하는걸 원치 않으신 분들은 \n다른 가죽으로도 수업 가능하니 메세지 주세요^^\n\n- 똑딱이 단추 스타일\n일반 골드, 니켈, 엔틱, 엔틱사틴, 그리고 싸개 단추 총 5가지가 있습니다.\n\n- 각인\n위드유레더에서는 불박기(기계)가 아닌 인두기로 \n수강생이 직접  가죽위에서 각인 하기에 불박기와는 다른,\n핸드메이드라는 느낌을 더욱 많이 받으실 수 있습니다.\n']
[79, 79, 32, 'EM발효액을 사용한 뽀득뽀득 설거지 비누 만들기', '70,000', 0, 0, '1 시간', '중', '서울 중랑구', '최대 3명', 0, 0, '요즘 제로 웨이스트, 레스 플라스틱 실천하시는 분들이 많아졌어요.\n환경을 위해 아주 좋은 움직임이죠:-)\n어렵게 생각하면 한없이 어려워지지만\n노력해 나가고자 하는 그 마음이 제일 중요하

[89, 89, 32, '천연탄산스파 (바스붐) 입욕제 1kg 만들기 클래스', '70,000', 0, 0, '2 시간', '중', '경기 파주시', '최대 4명', 0, 0, '\n🔶 소개\n\n나에게 맞는 바스붐을 만들어 가실 수 있는 클래스입니다.\n정제수와 알콜로 점도를 맞추지 않습니다.\n벨하이솝에서는 직접 제작한 자운고겔을 제공해 드립니다.\n자운고겔과 첨가물로 점도를 맞추어 제작하기때문에\n시중의 바스붐보다 훨씬 더 좋은 효과를 기대하실 수 있습니다.\n목적에 맞게, 사용 대상에 맞게, 취향에 맞게\n맞춤 바스붐을 직접 만들어 보실 수 있게 도와드려요 :)\n\n\n🔶 클래스 진행 순서 / 과정\n\n1. 가루계량\n2. 색상, 디자인 선택 \n3. 향 선택\n4. 반죽 만들기\n5. 틀에 넣어서 굳히기\n6. 건조 후 포장\n\n\n🔶 벨하이솝 클래스만의 특별한 점\n\n✔️ 탱자,소루쟁이,붉나무,어성초,병풀 등등 9종의 한약재 첨가물 공통 제공해드립니다.\n✔️ 집에서도 혼자서 충분히 만드실 수 있도록 레시피 제공해드립니다.\n\n\n🔶 알립니다\n\n계면활성제가 들어간 거품이 나는 버블바가 아닙니다.\n천연 탄산이 생기며 온천수 효과를 볼 수 있는 탄산스파 바스붐 만들기 클래스입니다.\n입욕물에 넣으면 거품대신 탄산이 생기며 피부 건강에 도움을 줍니다.\n버블바는 계면활성제때문에 사용 후 건조해질 수 있습니다. \n계면활성제free 천연탄산스파만 만들고 있으니 참고해주세요 :)\n\n\n✅원하시는 시간대가 옵션에 없는 경우 전화나 카카오톡 문의 주시면 예약 도와드리겠습니다 :)']
[90, 90, 32, '맞춤 수제 cp비누 / 모유비누 1kg 만들기 클래스', '100,000', 0, 0, '2 시간', '중', '경기 파주시', '최대 4명', 0, 0, '🔶 소개\n\n나에게 맞는 비누를 만들어 가실 수 있는 클래스입니다.\n정제수 외에 각종 증류수 부터 다양한 수상이 준비되어 있습니다.\n목적에 맞게, 사용 대상에 맞게, 취향에 맞게\

In [39]:
print(soup.find('ul',{'class':'img-list indicator'}))

<ul class="img-list indicator" data-size="5">
<li class="indicator-btn active" style="background-image: url(https://image.idus.com/image/files/66b9a9becb584145ad2d9ed82ff0435b_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/9719b4f6614742c39064a372ad17c2ec_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/e3d33da90c974f3390a0422023ab4b14_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/e23ef229e13f409aa46afb6eec67fafa_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/bf849362794041b5b8cbcc5357cdde6c_320.png)">
</li>
</ul>


In [38]:
print(soup.findAll('a',{"class":"ui_card__img"}))

[<a class="ui_card__img" data-lazy-img="https://image.idus.com/image/files/0ebb5fb5d77240ffb0a635cde59bc97c_320.png" href="/c/class/4272" style="background-image: url('https://image.idus.com/image/files/0ebb5fb5d77240ffb0a635cde59bc97c_320.png');" target="_blank">
</a>, <a class="ui_card__img" data-lazy-img="https://image.idus.com/image/files/ddad0ecb660f4dd39912288f89714b99_320.png" href="/c/class/4271" style="background-image: url('https://image.idus.com/image/files/ddad0ecb660f4dd39912288f89714b99_320.png');" target="_blank">
</a>, <a class="ui_card__img" data-lazy-img="https://image.idus.com/image/files/a195ff99451b4443b2f82f15cece00ed_320.png" href="/c/class/2432" style="background-image: url('https://image.idus.com/image/files/a195ff99451b4443b2f82f15cece00ed_320.png');" target="_blank">
</a>, <a class="ui_card__img" data-lazy-img="https://image.idus.com/image/files/c636e84339ac40b1a286d855716fe8ff_320.png" href="/c/class/4765" style="background-image: url('https://image.idus.com

In [66]:
with open ("ourdus_offline_class.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)

    # class_id , author_id,small_category_id, class_name, class_price, class_hit, class_purchase,
    # class_duration, class_level, class_place, user_max, class_like, class_rate,class_description
    myWriter.writerow(["class_id", "author_id","small_category_id", "class_name", "class_price", "class_hit", "class_purchase",
    "class_duration", "class_level", "class_place", "user_max", "class_like", "class_rate","class_description"])
    
    #1, 1, 12, '정성스럽고 격식있는 보자기 포장 매듭법 !', '80,000', 0, 0, '1.5 시간', '중', '경기 성남시', '최대 4명', 0, 0,
    
    for one in data:

        #클래스 가격 전처리
        if type(one[4])!=int:
            one[4]=one[4].replace(",","")
            one[4]=int(one[4])
        
        #클래스 소요기간 전처리
        if type(one[7])!=int:
            one[7] = float(one[7][0:-2])
            one[7]=int(one[7]*10)
        
        #클래스 수강인원 전처리
        if type(one[10])!=int:
            one[10]=one[10][-2:-1]
            one[10]=int(one[10])

        myWriter.writerow(one)
        print(one)

[1, 1, 12, '정성스럽고 격식있는 보자기 포장 매듭법 !', 80000, 0, 0, 15, '중', '경기 성남시', 4, 0, 0, '<  KNOTIST _ 노티스트 / 원데이 클래스  >\n\nº 1인 기준 : 8만원 / 매듭법 4가지\n(재료비 포함)\n\n\nº수업 시간 : 약 1시간 30분 \n(클래스 특성상 수업 시간이 더 소요될 수 있습니다.)\n\n\n- 보자기와 박스 및 재료들 4가지씩 제공됩니다.\n따로 준비하실 재료는 필기도구만 가져오시면 됩니다. !\n\n- 수업은 상시모집으로 일정 조율 후 진행됩니다.\n\n_____________________________\n\n\n노티스트 - 원데이 클래스에는\n\nº 다양한 보자기 색으로 여러 가지 매듭법을 배울 수 있으며, 의미가 담긴 선물에 정성과 격식을 담아 포장하실 수 있습니다.\n\nº 명절, 기념일 및 답례품과 같은 유용한 선물 포장법을 배워보고자 합니다.\n\nº 물건에 구애를 받지 않고 어떤 선물이든 격식있게 포장할 수 있습니다. ']
[2, 2, 12, '수제 전통계절주를 집에서 간단하게 빚어 건강하게 즐기자!', 48000, 0, 0, 20, '하', '서울 종로구', 6, 0, 0, '전통 발효 계절주 수업을 진행합니다.\n\n먼저 기초 이론 설명과 함께 시연을 하고...본인이 직접 만들어 보시고 ..완성품을 가져 가시는 방법입니다.\n\n1. 준비과정과 재료 설명\n\n2. 고두밥 짓기\n\n3. 직접 술빚기\n\n4. 완성 후 용기에 담기\n\n']
[3, 3, 32, '[3월의 데일리솝] 솜사탕컬러로 만드는 팜프리 비건비누 만들기', 50000, 0, 0, 20, '중', '서울 마포구', 2, 0, 0, '혹시 비건비누에 대해 알고 계신가요?\n\n팜프리비누라고도 불리며 비누를 만들면서 필수로 들어가는 팜유라는 식물성오일이 들어가지 않은 비누를 말해요.\n\n팜유는 비누의 단단함과 조밀한 거품, 안정적인 비누화를 위해 들어가지만 이 팜유가 환경오염과 관련이 되어있

In [68]:
print(imageList)
for i in imageList:
    print(i)
    print()

[[1, <ul class="img-list indicator" data-size="9">
<li class="indicator-btn active" style="background-image: url(https://image.idus.com/image/files/46a7be31ce5549588a6c9cc11d4f9336_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/220f1709b2c24ca997abc37c88b6b2cb_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/ffe1fb7781834b86a961783c75a671a9_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/07beb5c52fc34c0ab5a47da164225868_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/24d707435fbf427ab0a6478a4a5e1a6a_320.jpg)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/c80e194d6c184fba9101b9db9355baf9_320.png)">
</li>
<li class="indicator-btn" style="background-image: url(https://image.idus.com/image/files/31b4682ef21a

In [75]:
with open ("ourdus_offlineclass_image.csv", 'w', encoding='utf-8',newline="") as fileWrite:
    myWriter = csv.writer(fileWrite)

    # image_id,class_id,class_img
    myWriter.writerow(["image_id", "class_id","class_img"])
    img_id=0
    for image in imageList:
        class_id=image[0]
        imageUrls=image[1].findAll("li",{"class":"indicator-btn"})
        for url in imageUrls:
            img_id+=1
            url = url["style"]
            start=url.find('(')
            end = url.find(')',start+1)
            class_img = url[start+1:end]
            
            # 로컬 저장소에 사진 저장
            with urlopen(class_img) as f:
                with open('C:/Users/KSY/Desktop/offline_class_img/offline_class_'+ str(img_id)+ '.jpg',"wb") as h:
                    img=f.read()
                    h.write(img)
                    
            myWriter.writerow([img_id,class_id,class_img])
            print([img_id,class_id,class_img])

[1, 1, 'https://image.idus.com/image/files/46a7be31ce5549588a6c9cc11d4f9336_320.png']
[2, 1, 'https://image.idus.com/image/files/220f1709b2c24ca997abc37c88b6b2cb_320.png']
[3, 1, 'https://image.idus.com/image/files/ffe1fb7781834b86a961783c75a671a9_320.png']
[4, 1, 'https://image.idus.com/image/files/07beb5c52fc34c0ab5a47da164225868_320.png']
[5, 1, 'https://image.idus.com/image/files/24d707435fbf427ab0a6478a4a5e1a6a_320.jpg']
[6, 1, 'https://image.idus.com/image/files/c80e194d6c184fba9101b9db9355baf9_320.png']
[7, 1, 'https://image.idus.com/image/files/31b4682ef21a41428c203a2252b86ccb_320.png']
[8, 1, 'https://image.idus.com/image/files/0eb64cd9343547298b6f815ac428b778_320.png']
[9, 1, 'https://image.idus.com/image/files/2adff0fd6e3f4aa5acf56d1257805ff3_320.png']
[10, 2, 'https://image.idus.com/image/files/7497cacadd7b41fa9a724d432c0644dc_320.png']
[11, 2, 'https://image.idus.com/image/files/ed69511f187b4782ba563dc4847603ca_320.png']
[12, 2, 'https://image.idus.com/image/files/3d4cc6c9

[95, 17, 'https://image.idus.com/image/files/09cc750d29714783b4f95c766c6647c6_320.png']
[96, 17, 'https://image.idus.com/image/files/2fcb16f3b76741fdbcbec30865868fb7_320.png']
[97, 17, 'https://image.idus.com/image/files/4dda129305104be2bae00e1ff5955a63_320.png']
[98, 17, 'https://image.idus.com/image/files/c651e3e4903047e9ab9953b20ffc26ab_320.png']
[99, 17, 'https://image.idus.com/image/files/9f8c940b80494f3abdf942e7a93d4a2d_320.png']
[100, 17, 'https://image.idus.com/image/files/5520c1c7fa52485a8f853651b1025e25_320.png']
[101, 17, 'https://image.idus.com/image/files/f1a8a38897364a5583215e19834ddd41_320.png']
[102, 17, 'https://image.idus.com/image/files/92d3ba25c8c44911b715fae792d203fe_320.png']
[103, 18, 'https://image.idus.com/image/files/5d30ed60c50c41e98e9ae03860b06892_320.jpg']
[104, 18, 'https://image.idus.com/image/files/023ac56d454f4c2abba2d6fe2292c14d_320.jpg']
[105, 18, 'https://image.idus.com/image/files/f0156cc640b440a4b144cc17cff4ba3a_320.jpg']
[106, 18, 'https://image.i

[188, 32, 'https://image.idus.com/image/files/5d1c32ebb7e14571a2412ad4a0ad0587_320.png']
[189, 32, 'https://image.idus.com/image/files/3645e38c6262464ba5b5875393fc248c_320.png']
[190, 32, 'https://image.idus.com/image/files/cd48d0bd018d4ffcb2ba9fd34e21b8c1_320.png']
[191, 32, 'https://image.idus.com/image/files/4d73e57dab66491a82da7b8c0aea6d15_320.png']
[192, 32, 'https://image.idus.com/image/files/dbd6c9d41f5743dfbf09927a0af58df9_320.png']
[193, 32, 'https://image.idus.com/image/files/3aca07511fec4873b828f455ed3cc018_320.png']
[194, 32, 'https://image.idus.com/image/files/dc4eba9f3602457c8375f136b6a4d221_320.png']
[195, 33, 'https://image.idus.com/image/files/fbabd2a7b0c64b329aac0c2cd464687e_320.png']
[196, 33, 'https://image.idus.com/image/files/39e0f25fbf9d4d128dd530ee742e11f8_320.png']
[197, 33, 'https://image.idus.com/image/files/bf44ea5244f94681b7be3fcdb96cdb40_320.jpg']
[198, 33, 'https://image.idus.com/image/files/5e14ca0177ff4562ad7f602eeb3175fe_320.jpg']
[199, 33, 'https://im

[281, 45, 'https://image.idus.com/image/files/72044cd4d2294c3f9186c77e1af6fbb7_320.png']
[282, 46, 'https://image.idus.com/image/files/61a1bf41bfa645d8baed67f3887fe56b_320.png']
[283, 46, 'https://image.idus.com/image/files/911f4be19be74329a3f42e084fd14fd1_320.png']
[284, 46, 'https://image.idus.com/image/files/2b1a4170c9154349971b01a7f2c82e05_320.png']
[285, 46, 'https://image.idus.com/image/files/5aa4c12afe3f452cab27fdd67dd8cefb_320.png']
[286, 47, 'https://image.idus.com/image/files/345176fe111b474a89fe1bf74f0dffa7_320.png']
[287, 47, 'https://image.idus.com/image/files/a4218c061e344d92b9ffe986e0236bc1_320.png']
[288, 47, 'https://image.idus.com/image/files/b0b2b5cbc3494391a7b4102ceffab522_320.png']
[289, 47, 'https://image.idus.com/image/files/96add5cc271d43098f4923fe5e90bafa_320.png']
[290, 47, 'https://image.idus.com/image/files/2f01ec350a8541efa29396fede23e3a4_320.png']
[291, 47, 'https://image.idus.com/image/files/b5903ca6e28b42639702095fd61dd8eb_320.png']
[292, 48, 'https://im

[374, 60, 'https://image.idus.com/image/files/a5caad534f3146abb76573705c8c902f_320.jpg']
[375, 60, 'https://image.idus.com/image/files/b276864b97444acabf53efbbac2e525d_320.png']
[376, 61, 'https://image.idus.com/image/files/9c0b85fceeb1494e8772ca9f04088990_320.jpg']
[377, 61, 'https://image.idus.com/image/files/6dfef5dd33be498d853c0d2a7913d22e_320.jpg']
[378, 61, 'https://image.idus.com/image/files/c8eba60af0084e4e8efde5a42bfa937d_320.jpg']
[379, 61, 'https://image.idus.com/image/files/5f8d33ddc10b494c8c839f19c5aafd1a_320.jpg']
[380, 61, 'https://image.idus.com/image/files/b22c238673ce463bbe3c6af1b7f13e26_320.jpg']
[381, 61, 'https://image.idus.com/image/files/511a54a5065d48e69dc869234320446c_320.jpg']
[382, 61, 'https://image.idus.com/image/files/c7adcc3cf65c4143bdda0d5381301a82_320.jpg']
[383, 61, 'https://image.idus.com/image/files/ec22e7addbf24ba59487f864ade5cb5a_320.jpg']
[384, 61, 'https://image.idus.com/image/files/9f652a19a7a94d20855f7cd55681c830_320.jpg']
[385, 62, 'https://im

[467, 78, 'https://image.idus.com/image/files/f2f4e89772ca4e16ab3a156d7cdc7e24_320.jpg']
[468, 78, 'https://image.idus.com/image/files/cdfcfcd6cbba4bb6be26689f854b16c2_320.jpg']
[469, 78, 'https://image.idus.com/image/files/a342711d12164bccac880cff767e14cb_320.jpg']
[470, 78, 'https://image.idus.com/image/files/c7e9969efaeb42d986b9903f8ccde29c_320.jpg']
[471, 78, 'https://image.idus.com/image/files/40741ed6b656426d8e25d5273864bb2a_320.jpg']
[472, 78, 'https://image.idus.com/image/files/4d32306fc1344c418897d012bd5432b7_320.jpg']
[473, 78, 'https://image.idus.com/image/files/661104fb7fff4d11bfc8177c1209d227_320.jpg']
[474, 78, 'https://image.idus.com/image/files/79c638638fb641b690708e46f3fe959c_320.jpg']
[475, 78, 'https://image.idus.com/image/files/8a9d5cb1c0214f0d86baec9a1757d00f_320.jpg']
[476, 79, 'https://image.idus.com/image/files/965eddccb26b4bd4901df90f8770fd5a_320.jpg']
[477, 79, 'https://image.idus.com/image/files/a93f6768906e42c6be656e10373ed477_320.jpg']
[478, 79, 'https://im

[560, 91, 'https://image.idus.com/image/files/2877b9597e304e55b237ccf214d41adb_320.jpg']
[561, 91, 'https://image.idus.com/image/files/563cbac5b91d4c29b94179525d6026ab_320.jpg']
[562, 91, 'https://image.idus.com/image/files/9fbe6a84519e4cfb9d6752fb76ffb7ea_320.jpg']
[563, 91, 'https://image.idus.com/image/files/c6940929ccba4813b91a0cfbf93d5685_320.jpg']
[564, 91, 'https://image.idus.com/image/files/101c3b5a1460492c9b2936dd602f7c42_320.jpg']
[565, 92, 'https://image.idus.com/image/files/c7ccc5107fb24ffb8845b25d898dd974_320.jpg']
[566, 92, 'https://image.idus.com/image/files/346bc70d46664b6c92ce456da43cad90_320.jpg']
[567, 92, 'https://image.idus.com/image/files/4eeaded06c084727a873d59743f18cd2_320.jpg']
[568, 92, 'https://image.idus.com/image/files/f75dabde5170452fb115e09c205565a1_320.jpg']
[569, 92, 'https://image.idus.com/image/files/de9b3863e3b043c18ef6f6cfcbfb3ccd_320.jpg']
[570, 92, 'https://image.idus.com/image/files/62977b2c23ab46b986fb4afa8802121d_320.jpg']
[571, 93, 'https://im

In [74]:
print(class_img)

https://image.idus.com/image/files/68e277d3ba1b4f3ba42a22a4b6596e66_320.png
